In [1]:
import numpy as np
import pandas as pd
import string
import re
import nltk
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

In [2]:
d1 = pd.read_csv(r'C:\Users\shilp\OneDrive\Desktop\Cyberbullying\cyberbullying_tweets.csv')

stop_words = set(stopwords.words('english'))
def clear(text):
    text = re.sub(r'@[A-Za-z0-9]+','',text)
    text = re.sub(r'RT[\s]+','',text)
    text = re.sub(r'https?:\/\/\S+','',text)
    text = re.sub(r'#','',text)

    return text

d1['tweet_text'] = d1['tweet_text'].apply(clear)
d1['tweet_text'] = d1['tweet_text'].str.replace("[^a-zA-Z@]",' ')

d1['tweet_text'] = d1['tweet_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

tokenizer = RegexpTokenizer(r'\w+')
d1['tweet_text'] = d1['tweet_text'].apply(lambda x: tokenizer.tokenize(x.lower()))


stemmer = PorterStemmer()
def word_stemmer(text):
    stem_text = " ".join([stemmer.stem(i) for i in text])
    return stem_text

d1['tweet_text'] = d1['tweet_text'].apply(lambda x: word_stemmer(x))


lemmatizer = WordNetLemmatizer()
def word_lemmatizer(text):
    lem_text = [lemmatizer.lemmatize(i) for i in text]
    return lem_text

d1['tweet_text'].apply(lambda x: word_lemmatizer(x))


0        [o, t, h, e, r,  , w, o, r, d,  , k, a, t, a, ...
1        [a, u, s, s, i, e, t, v,  , w, h, i, t, e,  , ...
2        [c, l, a, s, s, i,  , w, h, o, r, e,  , m, o, ...
3        [_, g, i, o,  , m, e, h,  , t, h, a, n, k,  , ...
4        [t, h, i,  , i, s, i,  , a, c, c, o, u, n, t, ...
                               ...                        
47687    [b, l, a, c, k,  , a, r, e, n,  , t,  , e, x, ...
47688    [t, u, r, n, e, r,  , w, i, t, h, h, o, l, d, ...
47689    [s, w, e, a, r,  , g, o, d,  , t, h, i,  , d, ...
47690    [f, u, c, k,  , y, o, u, r,  , n, i, g, g, e, ...
47691    [b, r, o,  , g, o, t, t, a,  , c, h, i, l, l, ...
Name: tweet_text, Length: 47692, dtype: object

In [3]:
d1['tweet_text'].duplicated().sum()

3035

In [4]:
d1.drop_duplicates('tweet_text', inplace = True)

In [5]:
d1['cyberbullying_type'] = d1['cyberbullying_type'].replace(to_replace=['gender','other_cyberbullying','age','ethnicity'],value = 'Bullying')

In [6]:
d1['cyberbullying_type'].fillna(0,inplace=True)
d1['tweet_text'].fillna('',inplace=True)

In [7]:
X = d1['tweet_text']  
y = d1['cyberbullying_type'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)


tfidf = TfidfVectorizer(max_features = 5000)
X_train_tf = tfidf.fit_transform(X_train)
X_test_tf = tfidf.transform(X_test)  

print(X_train_tf.shape)
print(X_test_tf.shape)

(40191, 5000)
(4466, 5000)


In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
tfidf_array_train = X_train_tf.toarray()   # Converting the sparse matrix to a numpy array (dense matrix)
tfidf_array_test = X_test_tf.toarray()     # Converting the sparse matrix to a numpy array (dense matrix)
scaled_X_train = scaler.fit_transform(tfidf_array_train)  # Fitting on only training data to avoid data leakage from test data
scaled_X_test = scaler.transform(tfidf_array_test) # and then tranforming both training and testing data

In [9]:
import pickle

In [10]:
with open('tfidf_vectorization','wb') as f:
    pickle.dump(tfidf,f)

In [11]:
logic = LogisticRegression()
logic.fit(scaled_X_train,y_train)
#pred = logic.predict(X_test_tf)

c:\Users\shilp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [12]:
with open('model','wb') as f:
    pickle.dump(logic,f)

In [13]:
from sklearn.pipeline import Pipeline
pipe = Pipeline([('tfidf', TfidfVectorizer(max_features = 5000)), ('log', LogisticRegression())])
pipe.fit(X, y)

c:\Users\shilp\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('tfidf', TfidfVectorizer(max_features=5000)),
                ('log', LogisticRegression())])

In [14]:
pipe.predict(["hai you"])

array(['not_cyberbullying'], dtype=object)

In [19]:
pipe.predict(["i will kick you"])

array(['Bullying'], dtype=object)